In [1]:
import numpy as np
from tqdm.notebook import tqdm

from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import init_detector, inference_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

from xml_to_np import xml_to_np, all_xml_to_np
from AP import get_single_image_results
from soft_nms import py_cpu_softnms
from dbscan_result_to_aligned_result import dbscan_result_to_aligned_result
from utils import detect_quads, get_free_gpu

In [2]:
conf_threshold = 0.1
iou_threshold = 0.9

checkpoint_file="/home/aiarhipov/centernet/exps/32_quad_long/latest.pth"
config_file = "/home/aiarhipov/centernet/exps/32_quad_long/config.py"
cfg = Config.fromfile(config_file)
set_random_seed(0, deterministic=False)
dataset = build_dataset(cfg.data.test)


# model = init_detector(config_file, checkpoint_file, device=f"cpu")
model = init_detector(config_file, checkpoint_file, device=f"cuda:{get_free_gpu()}")
model.CLASSES = ("box",)

res = []
for idx in tqdm(range(int(len(dataset)))):
    anno = dataset.get_ann_info(idx=idx)
    boxes = anno["bboxes"]
    segm_path = anno["seg_map"]
    xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
    img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
    
    gt_boxes = all_xml_to_np(xml_path)

    quads = detect_quads(img_path, model, integer=False)
    quads = quads[quads[:, 8] > conf_threshold]
    
    boxes = np.vstack([(quads[:, 0] + quads[:, 6]) / 2, 
                       (quads[:, 1] + quads[:, 3]) / 2, 
                       (quads[:, 2] + quads[:, 4]) / 2, 
                       (quads[:, 5] + quads[:, 7]) / 2]).T
    
    nms_indexes = py_cpu_softnms(boxes, quads[:, 8], thresh=0.475, method=2)
    nms_quads = quads[nms_indexes]

    metrics = get_single_image_results(gt_boxes[:, :8], nms_quads[:, :8], iou_thr=iou_threshold, quad=True)
    
    if (metrics["true_pos"] + metrics["false_pos"]) != 0:
        metrics["Precision"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_pos"])
    else:
        metrics["Precision"] = 0
        
    if (metrics["true_pos"] + metrics["false_neg"]) != 0:
        metrics["Recall"] = metrics["true_pos"] / (metrics["true_pos"] + metrics["false_neg"])
    else:
        metrics["Recall"] = 0
        
    if (metrics["Precision"] + metrics["Recall"]) != 0:
        metrics["F1"] = 2 * metrics["Precision"] * metrics["Recall"] / (metrics["Precision"] + metrics["Recall"])
    else:
        metrics["F1"] = 0
        
    res.append(metrics)

    Precision = np.mean([d["Precision"] for d in res])
    Recall = np.mean([d["Recall"] for d in res])
    F1 = np.mean([d["F1"] for d in res])

    print(f"Precision = {Precision}, Recall = {Recall}, F1 = {F1}")
Precision = np.mean([d["Precision"] for d in res])
Recall = np.mean([d["Recall"] for d in res])
F1 = np.mean([d["F1"] for d in res])

print(f"Precision = {Precision}")
print(f"Recall = {Recall}")
print(f"F1 = {F1}")

loading annotations into memory...
Done (t=13.24s)
creating index...
index created!
Using GPU=3


2023-06-19 17:47:04,999 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-06-19 17:47:05,002 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-06-19 17:47:05,005 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /home/aiarhipov/centernet/exps/32_quad_long/latest.pth


  0%|          | 0/3611 [00:00<?, ?it/s]

/home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


Precision = 0.5, Recall = 0.25, F1 = 0.3333333333333333
Precision = 0.5911214953271028, Recall = 0.42418032786885246, F1 = 0.4854439592430859
Precision = 0.6825425353462737, Recall = 0.5712484237074401, F1 = 0.6120908446235958
Precision = 0.6695155971618793, Recall = 0.5665315558758182, F1 = 0.606276255295108
Precision = 0.68251513259676, Recall = 0.597573070787611, F1 = 0.6306350393238057
Precision = 0.6672474589821485, Recall = 0.5535331145452315, F1 = 0.596567450802625
Precision = 0.6811700908922618, Recall = 0.5815998124673413, F1 = 0.6195266083162583
Precision = 0.6982965568034563, Recall = 0.6311824446045757, F1 = 0.6534719208905874
Precision = 0.7076645722310916, Recall = 0.6190220763634876, F1 = 0.6504291470718264
Precision = 0.7218296218572975, Recall = 0.6267827900754535, F1 = 0.661929442241187
Precision = 0.7433299592642097, Recall = 0.631299862635439, F1 = 0.6738543518807029
Precision = 0.7538966185030148, Recall = 0.6443778152589563, F1 = 0.6866298637095744
Precision = 0.7